# Virus Tracking

The CDC tracks virus cases by state, where each state reports their current count to the CDC.
I want to see the number of cases in Texas as they grow over time, broken down by city if possible.

I have only found 1 website that offers anything like this kind of data, and it only shows the *current* cases:
https://dshs.texas.gov/news/updates.shtm

DSHS is updating their data source every day at noon, so if I *add* to my data source every day at 1pm (rather than replacing the current), I should be able to see change over time.

If you are curious, you can find the source for this report here:
* https://github.com/treytomes/covid-19-tracking/blob/master/Data%20Loader-arcgis.ipynb
* https://github.com/treytomes/covid-19-tracking/blob/master/Virus%20Tracking.ipynb
* https://github.com/treytomes/covid-19-tracking/blob/master/data.csv

As of around March 24th I've been getting my data from this site:
* https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83

It looks like the New York Times has a Github page with more extensive county-level data.  I plan on checking it against my own data before adopting it myself, but you can find that data here:
* https://github.com/nytimes/covid-19-data

In [14]:
# Step 0: Import all the things!

import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import geopy
from geopy.extra.rate_limiter import RateLimiter
from IPython.display import HTML, Markdown, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import urllib3

%run controls.ipynb

def datestamp(date):
    """Convert a datetime object into a date stamp: MM/DD/YYYY"""
    return f"{str(date.month).zfill(2)}/{str(date.day).zfill(2)}/{str(date.year).zfill(4)}"

today = dt.datetime.today()
today_text = datestamp(today)

yesterday = today - dt.timedelta(days=1)
yesterday_text = datestamp(yesterday)

# Now that data loading is done in a separate script, this should be all I need to load the data.
df_num_cases = pd.read_csv("data.csv")
df_num_cases = df_num_cases.drop(["Unnamed: 0"], axis=1)

min_date_text = df_num_cases["date"].min()
max_date_text = df_num_cases["date"].max()

parts = min_date_text.split("/")
min_date = dt.datetime(int(parts[2]), int(parts[0]), int(parts[1]))

parts = max_date_text.split("/")
max_date = dt.datetime(int(parts[2]), int(parts[0]), int(parts[1]))

total_num_counties = 254

# It's useful to have today's and yesterday's cases in their own DataFrames.
cases_today = df_num_cases.loc[df_num_cases["date"] == today_text]
cases_yesterday = df_num_cases.loc[df_num_cases["date"] == yesterday_text]

In [15]:
num_counties_added = len(cases_today) - len(cases_yesterday)

yesterday_counties = [row[0] for row in cases_yesterday.values]
today_counties = [row[0] for row in cases_today.values]

if num_counties_added > 0:
    display(Markdown(f"# *{num_counties_added}* counties were added from {yesterday_text} to {today_text}."))
    new_counties = [county for county in today_counties if county not in yesterday_counties]
elif num_counties_added < 0:
    display(Markdown(f"# *{-num_counties_added}* counties were **removed** from {yesterday_text} to {today_text}."))
    new_counties = [county for county in yesterday_counties if county not in today_counties]

new_counties = pd.DataFrame(new_counties)
new_counties.columns = ["county"]
display(new_counties)

# *1* counties were added from 04/10/2020 to 04/11/2020.

,county
0,Dimmit


In [16]:
total_output = widgets.Output()

def fct(date=dt.datetime.today()):
    date_text = datestamp(date) #f"{date.month}/{date.day}/{date.year}"
    
    prevdate = date - dt.timedelta(days=1)
    prevdate_text = datestamp(prevdate) #f"{prevdate.month}/{prevdate.day}/{prevdate.year}"
    
    date_total = df_num_cases.loc[df_num_cases["date"] == date_text]["num_cases"].astype('int32').sum()
    prevdate_total = df_num_cases.loc[df_num_cases["date"] == prevdate_text]["num_cases"].astype('int32').sum()
    
    delta_total = date_total - prevdate_total
    delta_total_text = "-" if delta_total < 0 else "+"
    delta_total_text += str(delta_total)
    
    num_counties_date = len(df_num_cases.loc[df_num_cases["date"] == date_text])
    
    with total_output:
        clear_output()
        #pd.options.display.max_rows = num_counties_date * 2
        #display(df_num_cases.loc[df_num_cases["date"] == date_text])
        display(Markdown(f"*{date_total}* cases found in Texas as of {date_text} ({delta_total_text} from yesterday)."))
        display(Markdown(f"{int((num_counties_date / total_num_counties) * 100)}% of Texas counties have confirmed cases."))

w=DatePicker(start=min_date_text,end=max_date_text,freq='D',fmt='%m/%d/%Y')
w.observe=fct
w.display()

display(total_output)

fct(max_date)

SelectionSlider(continuous_update=False, description='date', options=(('03/20/2020', Timestamp('2020-03-20 00:…

Output()

In [17]:
deltas = []
prevdate = min_date
currdate = prevdate + dt.timedelta(days=1)
while currdate <= max_date:
    currdate_text = datestamp(currdate)
    prevdate_text = datestamp(prevdate)
    
    currdate_total = df_num_cases.loc[df_num_cases["date"] == currdate_text]["num_cases"].astype('int32').sum()
    prevdate_total = df_num_cases.loc[df_num_cases["date"] == prevdate_text]["num_cases"].astype('int32').sum()
    
    deltas.append({
        "date": currdate_text,
        "cases_added": currdate_total - prevdate_total
    })
    
    prevdate = currdate
    currdate = prevdate + dt.timedelta(days=1)

df_deltas = pd.DataFrame(deltas)
display(df_deltas)

import altair as alt
alt.Chart(df_deltas, title="Cases Added per Day").mark_bar().encode(
    x=alt.X('date', bin=False),
    y='cases_added'
)

,date,cases_added
0,03/21/2020,110
1,03/22/2020,30
2,03/23/2020,18
3,03/24/2020,58
4,03/25/2020,564
5,03/26/2020,422
6,03/27/2020,335
7,03/28/2020,321
8,03/29/2020,500
9,03/30/2020,325


alt.Chart(...)

4/5/20: The average number of new cases per day is doubling about once a week.

4/6/20: Less cases reported today, but is that because they don't do any testing on Sunday?

4/10/20: Will we be at ~1200 new cases per day on April 14th?

In [18]:
# Everybody loves a good chart.  This one should become more useful over time.

import altair as alt

totals = pd.DataFrame({'num_cases' : df_num_cases.astype({'num_cases': 'int'}).groupby("date")["num_cases"].sum()}).reset_index()

alt.Chart(totals, title="The Curve to Flatten").mark_line().encode(
    x='date:O',
    y='num_cases:Q'
).properties(
  width=800
).interactive(bind_y=False)

alt.Chart(...)

So far the graph isn't exactly exponential, but it's not exactly linear either.  We're flattening the curve, but the exact slope of that the flattened curve is still being negotiated.

4/4/20: It's starting to get a little bit exponential.

In [19]:
county_name = "Nacogdoches"

deltas = []
prevdate = min_date
currdate = prevdate + dt.timedelta(days=1)
while currdate <= max_date:
    currdate_text = datestamp(currdate)
    prevdate_text = datestamp(prevdate)
    
    currdate_total = df_num_cases.loc[(df_num_cases["county"] == county_name) & (df_num_cases["date"] == currdate_text)]["num_cases"].astype('int32').sum()
    prevdate_total = df_num_cases.loc[(df_num_cases["county"] == county_name) & (df_num_cases["date"] == prevdate_text)]["num_cases"].astype('int32').sum()
    
    deltas.append({
        "date": currdate_text,
        "cases_added": currdate_total - prevdate_total
    })
    
    prevdate = currdate
    currdate = prevdate + dt.timedelta(days=1)

df_deltas = pd.DataFrame(deltas)
display(df_deltas)

import altair as alt
alt.Chart(df_deltas, title=f"Cases Added per Day ({county_name})").mark_bar().encode(
    x=alt.X('date', bin=False),
    y='cases_added'
)

display(df_num_cases.loc[(df_num_cases["county"] == county_name) & (df_num_cases["date"] == currdate_text)]["num_cases"].astype('int32').sum())

,date,cases_added
0,03/21/2020,0
1,03/22/2020,0
2,03/23/2020,0
3,03/24/2020,0
4,03/25/2020,0
5,03/26/2020,1
6,03/27/2020,0
7,03/28/2020,0
8,03/29/2020,1
9,03/30/2020,0


30

In [20]:
# Everybody loves a good chart.  This one should become more useful over time.

county_name = "Nacogdoches"
totals = pd.DataFrame({'num_cases' : df_num_cases.loc[df_num_cases["county"] == county_name].astype({'num_cases': 'int'}).groupby("date")["num_cases"].sum()}).reset_index()

alt.Chart(totals, title=f"The Curve to Flatten ({county_name})").mark_line().encode(
    x='date:O',
    y='num_cases:Q'
).properties(
  width=800
).interactive(bind_y=False)

alt.Chart(...)

In [23]:
import folium
from folium import plugins

def fct(date):
    county_map = folium.Map(
        location=[31.9686, -99.9018],
        tiles='cartodbpositron',
        zoom_start=6
    )
    
    date_text = datestamp(date)
    
    df_cases_date = df_num_cases.loc[df_num_cases["date"] == date_text]
   
    # This bit will put circles on the map.
    #df_num_cases.loc[df_num_cases["date"] == date_text].apply(lambda row: folium.CircleMarker(location=[row["latitude"], row["longitude"]], radius = row["num_cases"]).add_to(county_map), axis=1)
    
    plugins.HeatMap(data=df_cases_date[['latitude', 'longitude', 'num_cases']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=24, max_zoom=16).add_to(county_map)
    
    df_cases_date.apply(lambda row: folium.Marker(location=[row["latitude"], row["longitude"]],
                                                 icon=folium.DivIcon(html=f"<div style='font-size:1.3em'><b>{row['num_cases']}<b></div>")).add_to(county_map), axis=1)

    total_cases = sum([int(n) for n in df_num_cases.loc[df_num_cases["date"] == date_text]["num_cases"].to_list()])
    with map_output:
        clear_output()
        display(county_map)

w = DatePicker(start=min_date_text,end=max_date_text,freq='D',fmt='%m/%d/%Y')
w.observe = fct
w.display()

display(Markdown(f"This map shows the locations where virus has been found as of *{today_text}*."))

map_output = widgets.Output()
display(map_output)
fct(max_date)


SelectionSlider(continuous_update=False, description='date', options=(('03/20/2020', Timestamp('2020-03-20 00:…

This map shows the locations where virus has been found as of *04/11/2020*.

Output()

## One of the counties is showing up in Iowa.  Not sure which one.  I'll try to weed it out when the # cases for that region is more unique.

In [24]:
cases_delta = []
for row_today in cases_today.values:
    # Grab the num_cases from yesterday.
    row_yesterday = [n for n in cases_yesterday.values if n[0] == row_today[0]]
    if len(row_yesterday) == 0:
        delta_cases = row_today[2]
    else:
        row_yesterday = row_yesterday[0]
        delta_cases = int(row_today[2]) - row_yesterday[2]
    cases_delta.append([row_today[0], delta_cases, row_today[3], row_today[4]])

df_cases_delta = pd.DataFrame(cases_delta)
df_cases_delta.columns = ["county", "num_cases", "latitude", "longitude"]
df_cases_delta = df_cases_delta.loc[df_cases_delta["num_cases"] > 0]

county_map = folium.Map(
    location=[31.9686, -99.9018],
    tiles='cartodbpositron',
    zoom_start=6,
)

df_cases_delta.apply(lambda row: folium.CircleMarker(location=[row["latitude"], row["longitude"]], radius = row["num_cases"]).add_to(county_map), axis=1)

df_cases_delta.apply(lambda row: folium.Marker(location=[row["latitude"], row["longitude"]],
                                               icon=folium.DivIcon(html=f"<div style='font-size:1.3em'><b>{row['num_cases']}<b></div>")).add_to(county_map), axis=1)

display(Markdown("This map shows how much growth each area has had in the last 24 hours.  Circle size corresponds to the number of new cases."))
display(county_map)

This map shows how much growth each area has had in the last 24 hours.  Circle size corresponds to the number of new cases.

+216 in Houston alone on 3/29/20!

In [25]:
# TODO: It would be interesting to make a county map showing the average growth rate for each county in the last 7 days.

In [29]:
pd.options.display.max_rows = len(cases_today)
display(Markdown(f"# Cases were reported in each county as of {max_date_text}."))
display(cases_today)

# Cases were reported in each county as of 4/9/2020.

,county,date,num_cases,latitude,longitude
2180,Harris,4/9/2020,2341,29.811977,-95.374125
2181,Dallas,4/9/2020,1324,32.776272,-96.796856
2182,Travis,4/9/2020,597,30.287857,-97.756139
2183,Tarrant,4/9/2020,588,32.751366,-97.335696
2184,Bexar,4/9/2020,554,29.426399,-98.510478
2185,Fort Bend,4/9/2020,437,29.511218,-95.780735
2186,Denton,4/9/2020,398,33.183879,-97.141342
2187,Collin,4/9/2020,378,33.160963,-96.606098
2188,Galveston,4/9/2020,287,29.387225,-94.992736
2189,Lubbock,4/9/2020,214,33.563521,-101.879336
